In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%reload_ext autoreload

In [3]:
import logging
import logging.config
import configparser
import os

from pathlib import Path

# handle kill signals gracefully
import signal

# create a cache path to hold downloade album art
from cachepath import CachePath, TempPath, Path

# requests for downloading album art from remote
import requests
from requests import exceptions as rqExcept

# write out the requests file-like-object
import shutil

# clock 
from datetime import datetime
# loop delay - sleep
import time

# rate limit the queries on the LMS server
from ratelimiter import RateLimiter

# parse commandline arguments
import argparse

import lmsquery
from waveshare_epd import epd5in83

In [4]:
import cfg
import epdlib

In [5]:
# this works best as a global variable
logConfig = Path(cfg.LOGCONFIG)
logging.config.fileConfig(logConfig.absolute())
# logging.basicConfig(level=logging.DEBUG, format='%(asctime)s %(name)s %(levelname)s: %(message)s')

logger = logging.getLogger(__name__)

In [6]:
# logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger().setLevel(logging.INFO)

In [7]:
appShortName = 'slimpi'
name = 'com.txoof.'
appLongName = name+appShortName 

In [8]:
def read_config(cfgfile=None, required={}):
    if not cfgfile:
        return {}
    
    config = configparser.ConfigParser()
    logging.info(f'reading config file {cfgfile}')
    config.read(cfgfile)
    
    try:
        for section in required:
            if section not in config.sections():
                logging.debug(f'adding missing section: {section}')
                config.add_section(section)
        
            for option in required[section]:
                if not config.has_option(section, option):
                    logger.debug(f'adding missing option: {option}')
                    config[section][option] = str(required[section][option])
            with open(cfgfile, 'w') as file:
                config.write(file)
    except Exception as e:
        logging.exception(f'exception while writing config file: {e}')
        raise
    
    return config

In [9]:
def configuration(configFile=None):    
    
    # FIXME this is a terrible idea
    configDefaults = cfg.CONFIGDEFAULTS
    
    if not configFile:
         configFile = Path(cfg.CONFIGFILE)

    config = configparser.ConfigParser()
    logger.info(f'reading configuration: {configFile}')
    config.read(configFile)
    
    try:
        for section in configDefaults:
            if section not in config.sections():
                logger.debug(f'adding section: {section}')
                config.add_section(section)
            
            for option in configDefaults[section]:
                if not config.has_option(section, option):
                    logger.debug(f'missing option: {option}')
                    logger.debug(f'setting {option} to: {configDefaults[section][option]}')
                    config[section][option] = str(configDefaults[section][option])
            with open(configFile, 'w') as file:
                config.write(file)
    
    except Exception as e:
        logging.exception(f'exception on configuration file: {e}')
        raise
    
    logger.debug(f'config file contains sections: {config.sections()}')
    return config

In [10]:
# def query(lms, last=0, delay=7):
#     '''query the player only when a specified delay has passed
#     Accepts:
#         last: float - last time query was called

#     Returns:
#         tuple(last, lms.now_playing()
#     '''
#     if last==0:
#         last = time.clock_gettime(time.CLOCK_MONOTONIC)-delay

#     if time.clock_gettime(time.CLOCK_MONOTONIC) > last+delay:
#         return time.clock_gettime(time.CLOCK_MONOTONIC), lms.now_playing()

#     else:
#         return last, None

In [11]:
def cacheArt(query={}):
    if not query:
        return None
    
    cachePath = CachePath(appLongName, dir=True)
    required = ['artwork_url', 'album_id'] 
    missing = []
    
    for key in required:
        if not key in query:
            missing.append(key)
    if missing:
        logger.warning(f'required value(s) in `query` missing: {missing}')
        return None
    
    albumArtFile = cachePath/(query['album_id']+'.jpg')
    
    # check if file already exists in cache
    if albumArtFile.exists():
        logging.debug(f'album artwork already downloaded at: {albumArtFile}')
        return albumArtFile
    
    r = False
    albumArtURL = query['artwork_url']
    
    # try to fetch the album art
    try:
        r = requests.get(albumArtURL, stream=True)
    except requests.exceptions.RequestException as e:
        logging.error(f'failed to fetch artwork at {albumArtURL}: {e}')
        
    if r:
        try:
            with open(albumArtFile, 'wb') as outFile:
                shutil.copyfileobj(r.raw, outFile)
                logging.debug(f'wrote album art to: {albumArtFile}')
        except (OSError, FileExistsError, ValueError) as e:
            logging.error(f'failed to write {albumArtFile}: {e}')
    else:
        logging.error('skipping download of album artwork due to previous errors')

    return albumArtFile
    

In [12]:
class signalHandler(object):
    '''handle specific signals and allow graceful exiting while loop
    https://stackoverflow.com/a/31464349/5530152
    
    Signals Handled Gracefully:
        SIGINT
        SIGTERM
    Atributes:
        kill_now (bool) default: False
    '''
    kill_now = False
    def __init__(self):
        signal.signal(signal.SIGINT, self.exit_gracefully)
        signal.signal(signal.SIGTERM, self.exit_gracefully)
    
    def exit_gracefully(self, signum, frame):
        self.kill_now = True

In [13]:
def limit_callback(until):
        duration = int(round(until - time.time()))
        logging.debug(f'function call rate limited sleeping for {duration} seconds')

In [15]:
def arg_parse():
    parser = argparse.ArgumentParser()
    
    # configuration file
    parser.add_argument('-c', type=str, required=False,
                        help='use the specified configuration file. Default is stored in ~/.config/myApp/config.ini')
    
    
    
    args, unknown = parser.parse_known_args()
    logging.warn(f'unknwon commandline arguments: {unknown}')
    
    return args

In [16]:
def new_main():
    '''main entry point'''
    
    # set up the log level
    logger.info('starting program')
    
    
    
    config = configuration()

    sigHandler = signalHandler()

    # now playing fields to use
    fields = ['title', 'album', 'artist', 'mode', 'artwork_url']
    
    # lms server configuration
    player_id = config.get('server', 'player')
    lms = lmsquery.LMSQuery(player_id=player_id)
    
    # create the screen object for managing screen writing
    screen = epdlib.Screen()
    screen.epd = epd5in83.EPD()
#     screen.initEPD()

    # layout for music output - 
    # FIXME - this layout shouldn't be hard coded here
    music_layout = epdlib.Layout(layout=epdlib.layouts.threeRow)
#     music_layout.update_contents(query(lms, 0)[1])
    
    # layout for clock output
    # FIXME - this shouldn't be hard coded here
    clock_layout = epdlib.Layout(layout=epdlib.layouts.clock)
#     clock_layout.update_contents({'time': datetime.now().strftime("%H:%M"), 'mode': 'stop'})
    clock_layout.update_contents({'time': time_now(), 'mode': ' '})

    
    screen.initEPD()
    screen.elements=music_layout.blocks.values()
    
    refresh = False
    updated = 0
    last_update = 0
    
    nowplaying_id = None
    nowplaying_mode = None
    lmsQuery_ratelimit = RateLimiter(max_calls=1, period=3, callback=limit_callback)
    
    try:
        while not sigHandler.kill_now:
            response = None
            logging.debug('querying lms server')
            with lmsQuery_ratelimit:
                try:
                    response = lms.now_playing()
                except rqExcept.ConnectionError as e:
                    logging.warning(f'server could not find player id to query: {player_id}')
                    logging.warning('is the specified player active?')
                    response = {'title': f'Could not connect to meidia player: {player_id}',
                                'id': 'NONE', 'mode': 'ERROR'}
                    nowplaying_mode = response['mode']
                except KeyError as e:
                    logging.info(f'No playlist is active on player {player_id}')
                    response = {'title': 'No music is queued', 'id': 'NONE', 'mode': 'No Playlist'}
                    nowplaying_mode = response['mode']
            
            # check for a response
            if response:
                if response['id'] != nowplaying_id or response['mode'] != nowplaying_mode:
                    logging.info(f'track/mode change {response["mode"]}')
                    nowplaying_id = response['id']
                    nowplaying_mode = response['mode']
                    
                    albumArt = cacheArt(response)
                    if not albumArt:
                        albumArt = cfg.NOIMAGE
                    
                    # add the path to the album art into the response
                    response['coverart'] = str(albumArt)
                             
                    music_layout.update_contents(response)
                    refresh = music_layout
                else:
                    refresh = False
                    
            if nowplaying_mode != 'play' and screen.update.last_updated > 60:
                logging.debug('music appears to be paused')
                clock_layout.update_contents({'time': time_now(), 'mode': nowplaying_mode})
                refresh = clock_layout
#             else:
#                 refresh = False
            
            if refresh:
                logging.debug('refreshing display')
                screen.initEPD()
                screen.elements=refresh.blocks.values()
                screen.concat()
                screen.writeEPD()   
                refresh = False
            
            time.sleep(0.5)
    finally:
        logging.info("cleaning up and wiping screen")
        screen.initEPD()
        screen.clearEPD()

In [ ]:
import sys

In [ ]:
sys.argv.append('-c')
sys.argv.append('~/slimpi.ini')

In [17]:

new_main()

<ipython-input-16-de7239d93657>:new_main:5:INFO - starting program
<ipython-input-9-058caeef3ae2>:configuration:10:INFO - reading configuration: servercfg.ini
Screen:initEPD:167:INFO - <waveshare_epd.epd5in83.EPD object at 0xb0e5a470> initialized
<ipython-input-16-de7239d93657>:new_main:69:INFO - track/mode change pause
Screen:initEPD:167:INFO - <waveshare_epd.epd5in83.EPD object at 0xb0e5a470> initialized
<ipython-input-16-de7239d93657>:new_main:69:INFO - track/mode change play
Screen:initEPD:167:INFO - <waveshare_epd.epd5in83.EPD object at 0xb0e5a470> initialized
<ipython-input-16-de7239d93657>:new_main:69:INFO - track/mode change play
Screen:initEPD:167:INFO - <waveshare_epd.epd5in83.EPD object at 0xb0e5a470> initialized
<ipython-input-16-de7239d93657>:new_main:69:INFO - track/mode change play
Screen:initEPD:167:INFO - <waveshare_epd.epd5in83.EPD object at 0xb0e5a470> initialized
<ipython-input-16-de7239d93657>:new_main:69:INFO - track/mode change play
Screen:initEPD:167:INFO - <wav

In [ ]:
# if __name__ == '__main__':
    
#     foo = main()

In [17]:
sc = epdlib.Screen()
sc.epd = epd5in83.EPD()
sc.initEPD()
sc.clearEPD()

True